In [5]:
# create a client for the chroma vector store locally
import chromadb
chroma_client = chromadb.Client()
chroma_client.reset()


Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.
Exiting: Cleaning up .chroma directory


True

In [6]:
import os, openai

os.environ["OPENAI_API_TYPE"] = openai.api_type = "azure"
os.environ["OPENAI_API_VERSION"] = openai.api_version = "2022-12-01"

openai.api_base = os.environ["OPENAI_API_BASE"] = "ENTER ENDPOINT HERE"
openai.api_key = os.environ["OPENAI_API_KEY"] = "ENTER KEY HERE"

In [7]:
# define a function that will be used to embed documents when they are entered into the vector store
from chromadb.utils import embedding_functions
openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=os.environ["OPENAI_API_KEY"],
                model_name="text-embedding-ada-002"
            )

In [12]:
# we are going to use the directory loader which will leverage a local version of Unstructured-io / this can be replaced with any other loader or with Azure Form Recognizer
from langchain.document_loaders import DirectoryLoader
loader = DirectoryLoader('data/', glob="**/*.pdf")
docs = loader.load()


In [13]:
# check all the documents that were loaded
len(docs)

3

In [17]:
#split every document into chunks 
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(docs)

In [18]:
#generate embeddings for each chunk
import time, tqdm
embeddings = []

for text in tqdm.tqdm(texts):
    try:
        response = openai.Embedding.create(
            input=text.page_content,
            engine="text-embedding-ada-002")
        emb = response['data'][0]['embedding']
        embeddings.append(emb)
    except Exception as e:
        time.sleep(8)
        response = openai.Embedding.create(
            input=text.page_content,
            engine="text-embedding-ada-002")
        emb = response['data'][0]['embedding']
        embeddings.append(emb)

100%|██████████| 171/171 [00:51<00:00,  3.30it/s]


In [19]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# initialize the vector store and create a openai embedding function
azure_embeddings = OpenAIEmbeddings(document_model_name="text-embedding-ada-002",query_model_name="text-embedding-ada-002")
vectorstore = Chroma("my_collection", embedding_function=azure_embeddings)

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


In [20]:
# add the documents to the vector store
vectorstore._collection.add(
    ids= [f"doc_{i}" for i in range(len(texts))],
    documents=[texts[i].page_content for i in range(len(texts))],
    embeddings=embeddings,
    metadatas=[text.metadata for text in texts]
)

In [21]:
# we can create custom templates pass them in later

from langchain.prompts import PromptTemplate

prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. If there are multiple answers, describe them all.

{context}

Question: {question}
:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}


In [22]:
from langchain import VectorDBQAWithSourcesChain
from langchain.llms import AzureOpenAI

#initialize our chain
# THIS IS WHERE YOU CAN PASS YOUR PROMPT TEMPLATE !
chain = VectorDBQAWithSourcesChain.from_chain_type(llm=AzureOpenAI(deployment_name="davinci003", model_name="text-davinci-003"), chain_type="stuff", vectorstore=vectorstore)


In [23]:
# test the chain
answer = chain({"question": "What was the rate of infections for clarinex as advese events?"}, return_only_outputs=True)
print(answer)

{'answer': ' The rate of infections for clarinex as adverse events was 4.1% for CLARINEX Tablets 5 mg and 2.0% for placebo.\n', 'sources': 'data/c_clarinex_clarinex_pi.pdf'}


In [24]:
def return_answer(question):
    answer = chain({'question': question}, return_only_outputs=True)
    answer = f"""{answer['answer']}\n\n source: {answer['sources']}"""
    return answer

In [25]:
# small interface for Q/A against document set.
import gradio as gr 
with gr.Blocks() as demo:
    text = gr.Textbox(show_label=False, placeholder="Enter text and press enter").style(container=False)
    answerbox = gr.Textbox(show_label=False, placeholder="Answer").style(container=False)
    text.submit(return_answer, inputs=[text], outputs=[answerbox])
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
